In [7]:
import pandas as pd
import altair as alt
import os 
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
data_dir = project_dir.joinpath("data")
output_dir = data_dir.joinpath("output")

csv_file_path = output_dir.joinpath("clean_anon_survey.csv")
survey_df = pd.read_csv(csv_file_path, sep=';', dtype={'WLplants': str})
survey_df.set_index("id", inplace=True)

questions_csv = data_dir.joinpath('EN_questionlist.csv')
questions_df = pd.read_csv(questions_csv, sep=';')

# Countries Table

In [10]:
countries_df = survey_df[["ip_country",'Nemployees', 'Ncommunities', 'AREAtotal', 'AREAred', 'year']]
countries_df = countries_df.groupby(["ip_country"]).mean().round(1)
countries_df['Nanswers'] = survey_df.groupby(["ip_country"]).count()['seed']

survey_df.loc[survey_df['NCSOs'] < survey_df['NinstrumCSO'], 'NinstrumCSO'] = survey_df.loc[survey_df['NCSOs'] < survey_df['NinstrumCSO'], 'NCSOs']
survey_df['ShareNinstrumCSO'] = survey_df['NinstrumCSO'] / survey_df['NCSOs']
countries_df['SumNCSOs'] = survey_df.groupby(["ip_country"])['NCSOs'].sum(min_count=1)
countries_df['SumNinstrumCSO'] = survey_df.groupby(["ip_country"])['NinstrumCSO'].sum(min_count=1)
countries_df['AvgShareNinstrumCSO'] = survey_df.groupby(["ip_country"])['ShareNinstrumCSO'].mean().round(3)


survey_df['AREAtotal'] = survey_df['AREAtotal'].fillna(survey_df.groupby('ip_country')['AREAtotal'].transform('mean'))
countries_df['SumAREA'] = survey_df.groupby(["ip_country"])['AREAtotal'].sum(min_count=1) * 0.01 #km2

country_area_dict = {'BE': 30528, 'CH': 41291, 'FR': 543941, 'DE': 357581,'DK': 42947, 'ES': 498485, 'LU': 2586} #km2 https://en.wikipedia.org/wiki/List_of_European_countries_by_area
countries_df['country_area'] = countries_df.index.map(country_area_dict)
countries_df['Area_percent_covered'] = countries_df['SumAREA'] / countries_df['country_area']
countries_df['Share of CSOs with Sensors'] = countries_df['SumNinstrumCSO'] / countries_df['SumNCSOs']

column_names = ['Nanswers', 'Nemployees', 'Ncommunities', 'AREAtotal', 'SumAREA', 'AREAred','Area_percent_covered', 'SumNCSOs','SumNinstrumCSO', 'AvgShareNinstrumCSO', 'year', 'Share of CSOs with Sensors']
countries_df = countries_df.reindex(columns=column_names)
countries_df.rename(columns={'Nanswers':'Number of Answers', 'Nemployees': 'Avg number of employees', 'Ncommunities': 'Avg number of communities', 'AREAtotal': 'Avg catchment area (ha)', 'AREAred': 'Avg reduced catchment area (ha)'}, inplace=True)

output_file = output_dir.joinpath('countries_comparison_table.csv')
countries_df.to_csv(output_file, sep=';', encoding='utf-8-sig')
countries_df


,Number of Answers,Avg number of employees,Avg number of communities,Avg catchment area (ha),SumAREA,Avg reduced catchment area (ha),Area_percent_covered,SumNCSOs,SumNinstrumCSO,AvgShareNinstrumCSO,year,Share of CSOs with Sensors
ip_country,,,,,,,,,,,,
BE,1,1200.0,300.0,1350000.0,13500.000000,NaN,0.442217,4500.0,1600.0,0.356,1990.0,0.355556
CH,127,11.8,7.2,5214.1,6621.906839,378.8,0.160372,1792.0,722.0,0.487,1926.4,0.402902
DE,1,42.0,1.0,8600.0,86.000000,NaN,0.000241,3.0,3.0,1.000,1994.0,1.000000
DK,4,170.2,3.2,10166.8,406.670000,1227.0,0.009469,534.0,299.0,0.707,1972.2,0.559925
ES,4,935.5,1250028.5,13550.0,542.000000,4050.0,0.001087,213.0,134.0,0.832,1961.2,0.629108
FR,9,1912.2,19.9,18791.0,1691.190000,143.5,0.003109,838.0,198.0,0.370,1768.0,0.236277
LU,1,100.0,24.0,51461.0,514.610000,NaN,0.198998,238.0,100.0,0.420,1994.0,0.420168


# How many of their CSOs do organizations equip with sensors?

In [17]:
code = 'ShareNinstrumCSO'

binstep = 10

min = survey_df[code].min()
max = survey_df[code].max()
print(f"N = {survey_df.shape[0]}")
print(f"No CSOs monitored = {survey_df[survey_df[code] == 0].shape[0]}")
print(f"All CSOs monitored = {survey_df[survey_df[code] == 1].shape[0]}")
print(f"Some CSOs monitored = {survey_df[(survey_df[code] > 0) & (survey_df[code] < 1)].shape[0]}")

alt.Chart(survey_df, title='Shares of CSOs with Sensors').mark_bar().encode(
    alt.X(code, bin=alt.Bin(extent=[min, max], step=0.025)),
    y='count()',
).interactive()

N = 147
No CSOs monitored = 23
All CSOs monitored = 31
Some CSOs monitored = 72


alt.Chart(...)

# Wastewater Association Characteristics

To get an overview about the size of the wastewater associations in respective countries, Indicators for size such as catchment Area and Number of employees are plotted.

Question: is the measurement unit 'ha' used outside of DACH?

**Results**: 
- Swiss wastewater associations tend to have less than 100 employees and their catchment area is usually smaller than 10'000 ha. The largest catchment area is 80'000 ha and the highest number of employees 5'600.
- Other countries have typically more employees for similar catchment areas (maybe the answers for france were given in km2)
- The biggest wastewater association in terms of catchment area lies Belgium with a catchment area of 1'350'000 ha
- The wastewater association with most employees is in France with 9000 employees

In [ ]:
alt.Chart(survey_df).mark_circle(size=40).encode(
    x=alt.X('Nemployees:Q', axis=alt.Axis(values=[1,5,10,50,100,500,1000, 5000, 10000]), title='Number of employees (symlog)').scale(type='symlog'),
    # y=alt.Y('Ncommunities').scale(type='symlog'),
    y=alt.Y('AREAtotal:Q', axis=alt.Axis(values=[1,10,100,1000, 10000, 100000]), title='Catchment Area (Ha, symlog)').scale(type='symlog'),
    color=alt.Color('ip_country', title='Country'),
    tooltip=['ip_country','Nemployees', 'AREAtotal', 'Ncommunities', 'year', 'startlanguage']
).interactive()

alt.Chart(...)